# Install Nginx

Eine Anleitung findet man hier: https://www.raspberrypi.org/documentation/remote-access/web-server/nginx.md

Zunächst die Paketdatenbank aktualisieren. Danach das Paket nginx installieren. Die Option `--yes` kann auch durch `-y` ersetzt werden.

In [8]:
sudo apt-get update
sudo apt-get install --yes nginx-full

Holen:1 http://mirrordirector.raspbian.org/raspbian stretch InRelease [15,0 kB]
OK:2 http://archive.raspberrypi.org/debian stretch InRelease
OK:3 https://download.docker.com/linux/debian stretch InRelease
OK:4 https://deb.nodesource.com/node_6.x stretch InRelease
Es wurden 15,0 kB in 2 s geholt (6.354 B/s).
Paketlisten werden gelesen... Fertig
Paketlisten werden gelesen... Fertig
Abhängigkeitsbaum wird aufgebaut.       
Statusinformationen werden eingelesen.... Fertig
nginx-full ist schon die neueste Version (1.10.3-1+deb9u1).
0 aktualisiert, 0 neu installiert, 0 zu entfernen und 1 nicht aktualisiert.


Nun kann man auf http://pi-sense  (_pi-sense_ durch eigenen Hostname ersetzen) die Nginx Webseite besucht werden.

In [2]:
cat /var/www/html/index.nginx-debian.html

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [3]:
sudo sed -i 's/to nginx/to pi-sense/g' /var/www/html/index.nginx-debian.html

In [4]:
sudo sed -i 's/Debian/Respbian/g' /var/www/html/index.nginx-debian.html

In [5]:
sudo sed -i 's/debian/respbian/g' /var/www/html/index.nginx-debian.html

In [6]:
sudo sed -i 's/respbian.org/debian.org/g' /var/www/html/index.nginx-debian.html

## Konfiguration von nginx

In [7]:
cat /etc/nginx/nginx.conf

user www-data;
worker_processes auto;
pid /run/nginx.pid;
include /etc/nginx/modules-enabled/*.conf;

events {
	worker_connections 768;
	# multi_accept on;
}

http {

	##
	# Basic Settings
	##

	sendfile on;
	tcp_nopush on;
	tcp_nodelay on;
	keepalive_timeout 65;
	types_hash_max_size 2048;
	# server_tokens off;

	# server_names_hash_bucket_size 64;
	# server_name_in_redirect off;

	include /etc/nginx/mime.types;
	default_type application/octet-stream;

	##
	# SSL Settings
	##

	ssl_protocols TLSv1 TLSv1.1 TLSv1.2; # Dropping SSLv3, ref: POODLE
	ssl_prefer_server_ciphers on;

	##
	# Logging Settings
	##

	access_log /var/log/nginx/access.log;
	error_log /var/log/nginx/error.log;

	##
	# Gzip Settings
	##

	gzip on;
	gzip_disable "msie6";

	# gzip_vary on;
	# gzip_proxied any;
	# gzip_comp_level 6;
	# gzip_buffers 16 8k;
	# gzip_http_version 1.1;
	# gzip_types text/plain text/css application/json application/javascript text/xml application/xml application/xml+rss text/javascript;

	##
	#

Für uns ist zunächst die Zeilen `include /etc/nginx/conf.d/*.conf;` und `include /etc/nginx/sites-enabled/*;` wichtig.

Mit `include /etc/nginx/conf.d/*.conf;` werden alle Dateien mit der Endung __.conf__ im Pfad __/etc/nginx/conf.d/__ inkludiert.

Mit `include /etc/nginx/sites-enabled/*;` werden alle Dateien aus dem Ordner __/etc/nginx/sites-enabled__ inkludiert.

In [ ]:
ls -lachi /etc/nginx/conf.d

In [ ]:
ls -lachi /etc/nginx/sites-enabled

Hier finden wir einen Link zur Datei `/etc/nginx/sites-available/default`

In [ ]:
cat -n /etc/nginx/sites-available/default

Nach http://serverfault.com/questions/312494/hosting-folder-in-the-home-directory-using-nginx soll folgender Block im Abschnitt server eingefügt werden.

```
location ~ ^/~(.+?)(/.*)?$ {
    alias /home/$1/www$2;
    autoindex on;
}
```

Dies können wir mit dem Editor erledigen.

### Sed Ausflug

Hier das Einfügen mit `sed` und grep.

Das Suchen nach der Zeichenkette __#}__

__Achtung__ Das Zeichen # muss demaskiert werden

In [ ]:
grep -n \#} /etc/nginx/sites-available/default

Mit `grep` kann man nach einem Suchstring (RegEx) in einer Datei gesucht werden. Die Option `-n` gibt die Zeilen nummer aus.

In [ ]:
grep -n $'\t#}' /etc/nginx/sites-available/default

Ein Hack um nach Tabulatoren zu suchen. Siehe hier: http://stackoverflow.com/questions/1825552/grep-a-tab-in-unix

Das Kommando `sed` ohne die Option `-i` gibt die Datei auf der Konsole aus, ohne diese zu verändern.

In [ ]:
sed 's/\t#}/foo/' /etc/nginx/sites-available/default

Erwischt also alle zwei Vorkommen.

In [ ]:
sudo sed '0,/\t#}/s/\t#}/foo/' /etc/nginx/sites-available/default

Erwischt erstes Vorkommen (Informatiker zählen ab 0). Siehe hier: http://stackoverflow.com/questions/148451/how-to-use-sed-to-replace-only-the-first-occurrence-in-a-file

In [ ]:
sudo sed '1,/\t#}/s/\t#}/foo/' /etc/nginx/sites-available/default

In [ ]:
sudo sed '0,/\t#}/s/\t#}/\t#}\n\n\tlocation ~ ^\/~(.+?)(\/.*)?$ {\n\t\talias \/home\/$1\/www$2;\n\t\tautoindex on;\n\t}/' /etc/nginx/sites-available/default

In [ ]:
sudo sed -i '0,/\t#}/s/\t#}/\t#}\n\n\tlocation ~ ^\/~(.+?)(\/.*)?$ {\n\t\talias \/home\/$1\/www$2;\n\t\tautoindex on;\n\t}/' /etc/nginx/sites-available/default

Jetzt steht der Block drinnen. :-)

## Nginx neu starten

In [ ]:
sudo service nginx restart

Ein Besuch auf http://pi-sense/~pi liefert noch eine 404 Meldung.

In [ ]:
pwd

In [ ]:
mkdir /home/pi/www

Jetzt ist die 404 Meldung weg. Es kann nun eine Webseite erstellt werden.

In [ ]:
cp /var/www/html/index.nginx-debian.html /home/pi/www/index.html

In [ ]:
sudo sed -i 's/Welcome to pi-sense on Respbian!/Welcome to pi Homepage!/g' /home/pi/www/index.html

## PHP und Nginx

In [ ]:
sudo apt-get install --yes php5-fpm

In [ ]:
grep -n php /etc/nginx/sites-available/default

In [ ]:
sed -n '45,52p' /etc/nginx/sites-available/default

In [ ]:
sudo sed -i '45,48s/#//' /etc/nginx/sites-available/default

In [ ]:
sudo sed -i '50,52s/#//' /etc/nginx/sites-available/default

In [ ]:
sudo sed -i 's/index index.html/index index.php index.html/' /etc/nginx/sites-available/default

In [ ]:
sudo service nginx restart

In [ ]:
echo '<?php echo phpinfo(); ?>' | sudo tee /var/www/html/index.php